In [1]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv("/users/s1f102201582/projects/mhcc-moshi/.env")

/nfs1/s1f102201582/anaconda3/envs/mfa_unit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,4"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1000

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = True

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/data_stereo")
audio_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/data_stereo")

# mfa関連のパス
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")
mfa_input_dir = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/mfa_input")
mfa_output_dir = join(home_dir, "projects/mhcc-moshi/moshi/data/v1/mfa_output")

#RAGで読み取るPDFのパス
rag_pdf_dir = join(home_dir, "projects/mhcc-moshi/mental_docs/")

In [3]:
base_paths = [
    json_dir_path,
    audio_dir_path,
    mfa_input_dir,
    mfa_output_dir,
]

for p in base_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [4]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [5]:
loader = DirectoryLoader(
    rag_pdf_dir,
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cann

Loaded 3 documents


In [6]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [7]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [8]:
from typing import Literal

from pydantic import BaseModel, Field


class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [9]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [20]:
#promptを作成
import random

prompt_template = """あなたは、音声合成（TTS）用の対話シナリオを作成するプロのライターです。
以下の【要件】、【設定】、そして【指示】に従い、専門のカウンセラー (C) と クライアント (CL) のテキスト対話データを生成してください。

### 【目的】
生成するテキストは、音声合成エンジンで読み上げることを前提としています。そのため、台本のような流暢な書き言葉ではなく、人間同士が実際に会話する際の「話し言葉」を忠実に再現することを最優先とします。

### 【最重要要件：自然な会話の再現】
以下の要素を必ずテキストに含めてください。

1.  **極端に短い発話（ターン）の継続（最重要）:**
    * 一度の発話（セリフ）は、意図的に非常に短く区切ってください。
    * 人間同士の会話のように、短い言葉のキャッチボールが続くように構成してください。長い独白（モノローグ）や、一つの発話に多くの情報を詰め込むのは絶対に避けてください。

2.  **発話の途中の相槌（インターラプト）:**
    * これが最も重要です。**クライアント(CL)の発話**が少しでも長くなりそうな場合（例：10～15文字以上）、その**発話の途中に** **カウンセラー(C)**が「はい」「ええ」「うんうん」「なるほど」といった短い相槌を**独立した発話として**挟んでください。
    * クライアントは、その相槌を受けて話を続けるという、現実の会話で頻繁に起こる言葉の重なりや短いキャッチボールを忠実に再現してください。

3.  **クライアント(CL)の話し言葉の要素:**
    * 「よどみ（フィラー）」や「言い直し・ためらい」は、【指示】5で指定された内容に従って調整してください。

### 【設定】
* **話者:**
    * **C (カウンセラー):** 専門家。穏やかで、共感的・受容的なトーン。常に短い相槌を挟む。
    * **CL (クライアント):** 悩みを抱えている。

### 【指示】

1.  **指定された設定:**
    以下の指定された設定に基づいてシナリオを生成してください。
    * **クライアントのペルソナ:** {selected_persona}
    * **トピック:** {selected_topic}
    * **状況:** {selected_situation}
    * **クライアントの話し方:** {selected_style_name}

2.  **会話の開始ルール（名前の扱い）:**
    * シナリオは、必ず C (カウンセラー) がクライアントに呼びかける発話から始めてください。
    * 指定された【状況】が『初回』の場合: カウンセラーがクライアントの**名前（呼び方）を尋ねる発話**（例：「はじめまして。担当します〇〇です。まず、お名前（呼び方）を伺ってもよろしいですか？」）から始めてください。
    * 指定された【状況】が『初回』以外の場合: すでに名前は知っている設定とし、**名前を呼ばずに**（例：「こんにちは。その後の調子はいかがですか？」）始めてください。`〇〇さん`のようなプレースホルダーは**絶対に使用しないでください**。

3.  **会話の長さ:**
    * シナリオは、会話が**1分30秒〜2分程度**（テキスト量で約**8〜12往復程度**）続くようにしてください。

4.  **要件の遵守:**
    * 【最重要要件：自然な会話の再現】、特に「**1. 極端に短い発話**」と「**2. 発話の途中の相槌**」を厳密に守ってください。

5.  **クライアントの話し方の調整（指示）:**
    * 以下の指示に従って、**クライアント(CL)の発話**における話し方を調整してください。
    * **指示:** {selected_style_instruction}

6.  **ペルソナとトピックの統合（【ペルソナ】【トピック】変数に基づく）:**
    * 指定された【クライアントのペルソナ】（年齢・職業/立場）と【トピック】（悩み）を組み合わせて、クライアント(CL)の**具体的な会話内容や背景**を構築してください。
    * 例えば、ペルソナが「20代・会社員」でトピックが「職場の人間関係」なら、同僚や上司との関係に悩む若手社員の会話にしてください。
    * **重要:** クライアント自身が「私は〇〇です」のように、自分のペルソナを台詞で**明言しない**ようにしてください。あくまで会話の内容から**推測できる**ようにしてください。
"""

# トピックリスト（悩みの種類）(全40項目)
topic_list = [
  # 仕事・キャリア関連
  "仕事のプレッシャーや過労、バーンアウト",
  "職場の人間関係（上司、同僚、部下）",
  "キャリアプランの悩み、キャリアチェンジの不安",
  "転職・就職活動のストレス",
  "ハラスメント（パワハラ、モラハラなど）の影響",
  "仕事へのモチベーション低下、やる気が出ない",

  # 自己認識・感情関連
  "自己肯定感の低さ、自分を責めてしまう",
  "完璧主義、失敗への過度な恐れ",
  "他人の評価が過度に気になる、承認欲求",
  "劣等感（他人との比較）",
  "自分のやりたいことが分からない、アイデンティティの悩み",
  "感情のコントロールが難しい（怒り、イライラ、悲しみ）",
  "ネガティブ思考の癖、反芻思考（同じことをぐるぐる考えてしまう）",
  "焦燥感、何かに追われている感覚",
  "罪悪感（休むことへの罪悪感など）",
  "虚無感、むなしさ、生きがいを感じられない",
  "趣味や楽しみを感じられない（アンヘドニア）",
  
  # 対人関係
  "家族関係（親子、夫婦、兄弟、親戚）",
  "友人関係や恋愛関係の悩み",
  "コミュニケーションへの苦手意識（雑談、会議での発言など）",
  "人に頼ることができない、甘えられない",
  "他人の期待に応えすぎようとしてしまう（ピープルプリーザー）",
  "境界線（バウンダリー）の問題（NOと言えない）",
  "孤独感、疎外感",
  "HSP（繊細さ）に関する悩み",
  
  # 生活・健康関連
  "将来への漠然とした不安",
  "気分の落ち込み、無気力",
  "睡眠に関する悩み（寝付けない、途中で起きる、過眠）",
  "生活リズムの乱れ",
  "体調不良（頭痛、倦怠感、腹痛など）と気分の関連",
  "健康不安（病気への過度な心配）",

  # 習慣・行動関連
  "決断疲れ、何かを選ぶことができない",
  "先延ばし癖、物事が始められない",
  "SNS疲れ、デジタルデトックスの必要性",

  # 特定の出来事
  "ライフイベント（引っ越し、結婚、出産、育児、介護）に伴うストレス",
  "特定の出来事による軽いトラウマやフラッシュバック",
  "喪失体験（別れ、死別）からの回復（グリーフケア）",
  "過去の選択への後悔"
]

# 状況リスト（セッションの場面）
situation_list = [
  "初回",
  "初期",
  "中期（深掘り）",
  "中期（宿題の確認）",
  "中期（感情の表出）",
  "後期",
  "終了（終結）"
]

# クライアントのペルソナリスト（年齢と職業/立場のみ）
persona_list = [
    "20代・会社員",
    "30代・会社員",
    "40代・会社員",
    "50代・会社員",
    "20代・大学生",
    "20代・大学院生",
    "30代・管理職",
    "40代・主婦/主夫",
    "30代・フリーランス",
    "20代・アルバイト",
    "40代・パートタイム",
    "50代・自営業",
    "20代・求職中",
    "30代・育児休業中"
]

speaking_style_data = {
    "よどみが多い（ためらいがち）": "CLの発話に、フィラー（「えーと」「あのー」「なんていうか」「その…」）や「言い直し・言いよどみ」を意図的に【多く】挿入してください。言葉がすぐに出てこない、ためらいながら話す様子を強く表現してください。",
    "普通（自然な会話）": "【最重要要件】の指示通り、自然な会話の範囲で適度なフィラーと「言い直し」を挿入してください。",
    "スムーズ（よどみ少なめ）": "CLの発話におけるフィラーや「言い直し・言いよどみ」を意図的に【最小限に】し、比較的流暢に話すようにしてください。（ただし、「1. 極端に短い発話」のルールは守り続けてください）"
}

def gen_prompt_txt():
    # ランダムに選択
    selected_topic = random.choice(topic_list)
    selected_situation = random.choice(situation_list)
    selected_persona = random.choice(persona_list)
    selected_style_name = random.choice(list(speaking_style_data.keys()))
    selected_style_instruction = speaking_style_data[selected_style_name]

    # print("選ばれたトピック:", selected_topic)
    # print("選ばれた状況:", selected_situation)
    # print("選ばれたペルソナ:", selected_persona)
    # print("選ばれたクライアントのスタイル:", selected_style_name)

    # 変数をプロンプトに埋め込む
    final_prompt = prompt_template.format(
        selected_persona=selected_persona,
        selected_topic=selected_topic,
        selected_situation=selected_situation,
        selected_style_name=selected_style_name,
        selected_style_instruction=selected_style_instruction
    )
    
    return final_prompt

In [25]:
# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [26]:
dialogues_list = gen_txt_dialogue()
for dial in dialogues_list:
    print(f"{dial.speaker}: {dial.text}")

選ばれたトピック: 家族関係（親子、夫婦、兄弟、親戚）
選ばれた状況: 初期
選ばれたペルソナ: 30代・管理職
選ばれたクライアントのスタイル: よどみが多い（ためらいがち）
A: はじめまして。
A: 担当しますカウンセラーの〇〇です。
A: まず、お名前を…
A: どのようにお呼びすればよろしいでしょうか？
B: あ、はい。
B: 〇〇と申します。
B: あの…下の名前で…
B: 〇〇って呼んでいただけたら…。
A: 〇〇さんですね。
A: 承知いたしました。
A: 今日は、その…
A: どのようなことでいらっしゃいましたか？
B: えーと…
B: なんていうか、こう…
B: 最近…
B: その、家族のことで…
B: 色々ありまして…
A: はい。
B: うーん…
B: 特に、その…
B: 親との関係で…
B: 色々、なんていうか…
A: はい。
B: その…思うところがありまして…
A: うんうん。
B: 仕事も、えーと…
B: 今、管理職なので…
A: なるほど。
B: その…忙しいんですけど…
B: 親から…
B: なんていうか…
A: はい。
B: 「もっとこうしろ」みたいな…
A: ご期待があるんですね。
B: はい…
B: なんか…
B: 期待が、えーと…
B: なんていうか、こう…
B: すごく…
B: 重荷に感じることが…
A: うんうん。
B: 増えてきてしまって…
A: 具体的に、その…
A: 「もっとこうしろ」というのは…
A: どのようなことなんでしょうか？
B: あのー…
B: 例えばなんですけど…
B: 結婚のこととか…
B: 孫の顔が見たいとか…
A: ええ。
B: そういう、なんていうか…
B: 私自身の…
B: プライベートな部分にまで…
A: なるほど。
B: 口出しされることが多くて…
A: それが、その…
A: 管理職としてのお仕事の忙しさもあって…
A: 少し…
A: 負担に感じていらっしゃる、と。
B: はい…
B: その通りです…
B: どうしたらいいのか…
B: わからなくて…
